In [ ]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller
import pandas as pd
from selenium import webdriver
from datetime import datetime
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings('ignore')

from selenium.common.exceptions import NoSuchElementException
import ping3
import time

In [ ]:
# page crawling function
def page_crawling(browser, date):
    temp_df = pd.DataFrame(columns=['date', 'journal', 'title', 'url'])
    idx = 0
    for num in range(1, 10+1):
        try:
            journal = browser.find_element(By.XPATH, f'//*[@id="rso"]/div/div/div[{num}]/div/div/a/div/div[2]/div[1]/span').text
            title = browser.find_element(By.XPATH, f'//*[@id="rso"]/div/div/div[{num}]/div/div/a/div/div[2]/div[2]').text
            url = browser.find_element(By.XPATH, f'//*[@id="rso"]/div/div/div[{num}]/div/div/a').get_attribute('href')

            temp_df.loc[idx] = [date, journal, title, url]
            idx += 1
            
        except NoSuchElementException:
            break

    
    return temp_df

In [ ]:
# chrome driver setting
options = webdriver.ChromeOptions()
service = Service(executable_path=chromedriver_autoinstaller.install(path = os.getcwd()))

options.add_argument('--no-sandbox') # 샌박끔        
options.add_argument('--disable-dev-shm-usage') # /dev/shm 비활
options.add_argument('headless') # 헤드리스
options.add_argument('--blink-settings=imagesEnabled=false') # 이미지 사용 안함
options.add_argument('--mute-audio') # 음소거
options.add_argument('disable-gpu')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36')


browser = webdriver.Chrome(options=options, service=service)


In [ ]:
# hyper parameter

keyword = 'apple+stock'
start_date = '2020-01-01'
end_date = '2024-01-31'
duration = pd.date_range(start_date, end_date, freq='D').strftime('%Y%m%d').tolist()


df = pd.DataFrame(columns=['date', 'journal', 'title', 'url'])

In [ ]:
omitted_list = sorted(list(set(duration) - set(df['date'].unique())))

In [ ]:
cnt = 0
while omitted_list:
    pbar = tqdm(omitted_list, 
                desc = 'crawling_complete', 
                total = len(omitted_list),
                leave = True)


    hostname = 'www.google.com'

    for date in pbar:
        response = ping3.ping(hostname)

        if response is None:
            print("인터넷 연결상태 불량, 2분 대기 후 다시 시도합니다.")
            time.sleep(100)
        
        browser.get(f'https://www.google.com/search?q={keyword}&sca_esv=0bc39af9d1d15b37&hl=en&biw=1024&bih=1029&prmd=znivsmb&sxsrf=ACQVn0_vMs2i3NIhDdmaUW6UANsHeBObUg%3A1713107930029&source=lnt&tbs=cdr%3A1%2Ccd_min%3A{date}%2Ccd_max%3A{date}&tbm=nws')
        browser.implicitly_wait(10)
        # page crawling
        result = page_crawling(browser, date)

        df = pd.concat([df, result], axis=0)


    pbar.close()
    df.to_csv(f'../data/google_news_crawling_omitted.csv', index=False, encoding='utf-8-sig')

    omitted_list = sorted(list(set(duration) - set(df['date'].unique())))
    print(f'누락 날짜 수 : {len(omitted_list)}')
    cnt += 1


    if cnt ==5: break


browser.quit()